In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation
from os import path, walk, listdir
from pprint import pprint
from datetime import datetime

In [2]:
BASE_DIR = './data'
PRICE_DIR = path.join(BASE_DIR, 'daily_price')
SIC_DIR = path.join(BASE_DIR, 'sic4_codes')

In [15]:
df_all_prices = pd.DataFrame()
for f in listdir(PRICE_DIR):
    p = path.join(PRICE_DIR,f)
    if path.isfile(p):
        try:
            df = pd.read_csv(p)
            df['composite_ticker'] = f.replace('_daily_price.csv', '').replace('_', ':')
            df_all_prices = pd.concat([df_all_prices, df], ignore_index=True)
        except pd.errors.EmptyDataError:
            print(p)

./data\daily_price\BIIB V_US_daily_price.csv
./data\daily_price\BLZ_US_daily_price.csv
./data\daily_price\C.VS_US_daily_price.csv
./data\daily_price\CTXS V_US_daily_price.csv
./data\daily_price\EKD_US_daily_price.csv
./data\daily_price\FCX A_US_daily_price.csv
./data\daily_price\GE4317671_US_daily_price.csv
./data\daily_price\HLT WI_US_daily_price.csv
./data\daily_price\HON WI_US_daily_price.csv
./data\daily_price\JPL.E_US_daily_price.csv
./data\daily_price\JPM3995814_US_daily_price.csv
./data\daily_price\SPC_US_daily_price.csv
./data\daily_price\VAR WI_US_daily_price.csv


In [16]:
df_all_prices.head()

,date,intraperiod,frequency,open,high,low,close,volume,adj_open,adj_high,adj_low,adj_close,adj_volume,composite_ticker
0,2005-12-30,False,daily,37.55,37.58,36.92,37.14,463271.0,35.394313,35.422591,34.800481,35.007851,463271.0,AAL:US
1,2005-12-29,False,daily,37.45,37.95,37.19,37.38,508847.0,35.300054,35.771350,35.054980,35.234073,508847.0,AAL:US
2,2005-12-28,False,daily,38.00,38.15,36.35,37.45,878820.0,35.818480,35.959868,34.263204,35.300054,878820.0,AAL:US
3,2005-12-27,False,daily,37.99,38.80,37.20,37.46,1277212.0,35.809054,36.572553,35.064406,35.309480,1277212.0,AAL:US
4,2005-12-23,False,daily,36.73,38.00,36.73,37.73,1321716.0,34.621388,35.818480,34.621388,35.563980,1321716.0,AAL:US


In [17]:
df_company_info = pd.read_csv(path.join(BASE_DIR, 'company_info.csv'))

In [18]:
df_company_info.head()

,id,ticker,name,lei,legal_name,stock_exchange,sic,short_description,long_description,ceo,...,entity_status,sector,industry_category,industry_group,template,standardized_active,first_fundamental_date,last_fundamental_date,first_stock_price_date,last_stock_price_date
0,com_NX6GzO,AAPL,Apple Inc,HWUPKR0MPOU8FGXBT394,APPLE INC,NASDAQ,3571,"Apple, Inc. engages in the design, manufacture...","Apple Inc. designs, manufactures, and markets ...",Tim Cook,...,ACTIVE,Consumer Goods,Consumer Durables,Electronic Equipment,indu,True,2007-09-29,2020-03-28,1980-12-12,2020-07-24
1,com_2zvNzA,XOM,Exxon Mobil Corp,J3WHBG0MTS7O8ZVMDC91,EXXON MOBIL CORP,NYSE,2911,"Exxon Mobil Corp. engages in the exploration, ...",Exxon Mobil Corporation explores for and produ...,Darren W. Woods,...,ACTIVE,Basic Materials,Energy,Major Integrated Oil & Gas,indu,True,2007-12-31,2020-03-31,1970-01-02,2020-07-24
2,com_0XLDXx,ORCL,Oracle Corp,1Z4GXXU7ZHVWFCD8TV52,ORACLE CORP,NYSE,7372,Oracle Corp. provides enterprise software and ...,"Oracle Corporation develops, manufactures, mar...",Larry Ellison,...,ACTIVE,Technology,Computer Software & Services,Application Software,indu,True,2008-05-31,2020-05-31,1986-03-12,2020-07-24
3,com_VXWJgv,KO,Coca-Cola Co,UWJKFUJFZ02DKWI3RY53,COCA COLA CO,NYSE,2080,The Coca-Cola Co. operates as a beverage compa...,"The Coca-Cola Company, a beverage company, man...",Ahmet Muhtar Kent,...,ACTIVE,Consumer Goods,Food & Beverage,Beverages - Soft Drinks,indu,True,2007-12-31,2020-06-26,1962-01-02,2020-07-24
4,com_NgYGzd,INTC,Intel Corp,KNX4USFCNGPY45LOCE31,INTEL CORP,NASDAQ,3674,"Intel Corp. designs, manufactures and sells co...","Intel Corporation designs, manufactures, and s...",Brian M. Krzanich,...,ACTIVE,Technology,Electronics,Semiconductor - Broad Line,indu,True,2007-12-29,2020-06-27,1980-03-17,2020-07-24


In [43]:
df_company_info['company_id'] = df_company_info.id
df_company_info = df_company_info.drop(columns=['id'])

In [19]:
df_securities = pd.read_csv(path.join(BASE_DIR, 'securities_list.csv'))

In [20]:
df_securities.head()

,id,company_id,name,code,currency,ticker,composite_ticker,figi,composite_figi,share_class_figi
0,sec_agjrgj,com_NX6GzO,Apple Inc,EQS,USD,AAPL,AAPL:US,BBG000B9Y5X2,BBG000B9XRY4,BBG001S5N8V8
1,sec_gkmNYz,com_2zvNzA,Exxon Mobil Corp.,EQS,USD,XOM,XOM:US,BBG000GZQBJ1,BBG000GZQ728,BBG001S69V32
2,sec_gPV3vz,com_0XLDXx,Oracle Corp.,EQS,USD,ORCL,ORCL:US,BBG000BQLY10,BBG000BQLTW7,BBG001S5SJG6
3,sec_X7m9Zy,com_VXWJgv,Coca-Cola Co,EQS,USD,KO,KO:US,BBG000BMX4N8,BBG000BMX289,BBG001S5SMQ8
4,sec_zvNxEz,com_NgYGzd,Intel Corp.,EQS,USD,INTC,INTC:US,BBG000C0GFS4,BBG000C0G1D1,BBG001S5SF65


In [44]:
df_securities['securities_id'] = df_securities.id
df_securities = df_securities.drop(columns=['id'])

In [47]:
df_company_securities = pd.merge(
    df_securities[['securities_id', 'company_id', 'name', 'ticker', 'composite_ticker']],
    df_company_info[['company_id', 'stock_exchange', 'sector', 'industry_category', 'industry_group']],
    on=['company_id'], how='left'
)

In [48]:
df_company_securities

,securities_id,company_id,name,ticker,composite_ticker,stock_exchange,sector,industry_category,industry_group
0,sec_agjrgj,com_NX6GzO,Apple Inc,AAPL,AAPL:US,NASDAQ,Consumer Goods,Consumer Durables,Electronic Equipment
1,sec_gkmNYz,com_2zvNzA,Exxon Mobil Corp.,XOM,XOM:US,NYSE,Basic Materials,Energy,Major Integrated Oil & Gas
2,sec_gPV3vz,com_0XLDXx,Oracle Corp.,ORCL,ORCL:US,NYSE,Technology,Computer Software & Services,Application Software
3,sec_X7m9Zy,com_VXWJgv,Coca-Cola Co,KO,KO:US,NYSE,Consumer Goods,Food & Beverage,Beverages - Soft Drinks
4,sec_zvNxEz,com_NgYGzd,Intel Corp.,INTC,INTC:US,NASDAQ,Technology,Electronics,Semiconductor - Broad Line
...,...,...,...,...,...,...,...,...,...
859,sec_Qz8jlg,com_gemwlg,Cigna Corp.,CI,CI:US,NYSE,Healthcare,Health Services,Health Care Plans
860,sec_gxVQ6k,com_XZqpQg,Fox Corporation,FOXA,FOXA:US,NASDAQ,Services,Media,Entertainment - Diversified
861,sec_gxVQ6k,com_XZqpQg,Fox Corporation,FOXA,FOXA:US,NASDAQ,Services,Media,Entertainment - Diversified
862,sec_gBAKOe,com_zENABy,Otis Worldwide Corp,OTIS,OTIS:US,NYSE,Technology,Electronics,Industrial Electrical Equipment


In [50]:
df_company_securities.to_csv(path.join(BASE_DIR, 'test.csv'), index=False)

In [52]:
len(df_company_securities.ticker.unique())

827

In [54]:
df_company_securities.groupby('sector').securities_id.count()

sector
Basic Materials      52
Consumer Goods       68
Financial           382
Healthcare           65
Industrial Goods     53
Services            109
Technology           94
Utilities            35
Name: securities_id, dtype: int64

In [61]:
df_securities_distribution = df_company_securities.groupby(['sector', 'industry_category', 'industry_group']).securities_id.count().reset_index()

In [63]:
df_securities_distribution.to_csv(path.join(BASE_DIR, 'distribution.csv'), index=False)